In [4]:
import numpy as np
from numpy import asarray
import tensorflow as tf
import tensorflow_datasets as tfds
import glob
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import os
from sklearn.model_selection import StratifiedKFold
import pathlib

In [11]:
#count total number of images
data_dir = pathlib.Path("../input/vehicleImage/vehicles")
image_count = len(list(data_dir.glob('*/*.JPEG')))
print(image_count)

#get the list of names of all the images and shuffle
list_ds = tf.data.Dataset.list_files("../input/vehicleImage/vehicles/*/*", shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [14]:
for f in list_ds.take(4):
  print(f.numpy())

In [18]:
#The tree structure of the files can be used to compile a class_names list.
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [20]:
#Split the dataset into training and validation sets:
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

In [27]:
# functions that convert a file path to an (img, label) pair:
batch_size = 32
img_height = 500
img_width = 500

def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [28]:
# Use Dataset.map to create a dataset of image, label pairs:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [29]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [30]:
#Configure dataset for performance
#To be well shuffled.
#To be batched.
#Batches to be available as soon as possible.

def configure_for_performance(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

In [32]:
#Visualize the data
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].numpy().astype("uint8"))
  label = label_batch[i]
  plt.title(class_names[label])
  plt.axis("off")

In [35]:
num_classes = 10

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [37]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
#training the model
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)


In [ ]:
#skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=5)
skf = StratifiedKFold(n_splits=3)
for train_index, test_index in skf.split(Images, targets):
    targets_train, targets_test = np.array(targets)[train_index], np.array(targets)[test_index]
    Images_train, Images_test = np.array(Images)[train_index], np.array(Images)[test_index]
    

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Aircraft_Carrier, labels))
#dataset = dataset.batch(32)
#    out_images = np.array(X_train)[indices.astype(int)]
    

In [ ]:
pyplot.imshow(Images[0])
pyplot.show()
pyplot.imshow(Images[1])
pyplot.show()
pyplot.imshow(Images[2])
pyplot.show()


In [ ]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

In [ ]:
# Load and preprocess images
Aircraft_Carrier = []
for filename in glob.glob('../input/vehicleImage/vehicles/Aircraft_Carrier/*.JPEG'): #assuming JPEG
    # load the image and resize
    im=Image.open(filename).resize((500,500))
    imRGB=im.convert('RGB')
    # convert image to numpy array
    imdata = asarray(imRGB)
    Aircraft_Carrier.append(imdata)
    
    

In [ ]:
print("Original dataset")

#for i in dataset:   
#    print(i)

In [ ]:
from sklearn.model_selection import train_test_split
# set aside 20% of train and test data for evaluation

X_train, X_test, y_train, y_test = train_test_split(
ac, lbs, test_size=0.2, random_state=42)
#np.split(x, 3)


In [ ]:
print("Original dataset")
y_test

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(train, test,
    test_size=0.2, shuffle = True, random_state = 8)


# step 1
filenames = tf.constant(['im_01.jpg', 'im_02.jpg', 'im_03.jpg', 'im_04.jpg'])
labels = tf.constant([0, 1, 0, 1])

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

dataset = dataset.map(_parse_function)
dataset = dataset.batch(2)

# step 4: create iterator and final input tensor
iterator = dataset.make_one_shot_iterator()
images, labels = iterator.get_next()


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
# step 1
filenames = tf.constant(['im_01.jpg', 'im_02.jpg', 'im_03.jpg', 'im_04.jpg'])
labels = tf.constant([0, 1, 0, 1])

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

dataset = dataset.map(_parse_function)
dataset = dataset.batch(2)

# step 4: create iterator and final input tensor
iterator = dataset.make_one_shot_iterator()
images, labels = iterator.get_next()






In [ ]:
# Load and preprocess images
import numpy as np
from numpy import asarray
import os
import PIL
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
import glob
from matplotlib import pyplot

Aircraft_Carrier = []
for filename in glob.glob('../input/vehicleImage/vehicles/Aircraft_Carrier/*.JPEG'): #assuming JPEG
    # load the image and resize
    im=Image.open(filename).resize((500,500))
    imRGB=im.convert('RGB')
    # convert image to numpy array
    imdata = asarray(imRGB)
    Aircraft_Carrier.append(imdata)

# summarize shape
print(Aircraft_Carrier[0].shape)
print(Aircraft_Carrier[1].shape)
print(Aircraft_Carrier[2].shape)
print(Aircraft_Carrier[3].shape)

pyplot.imshow(Aircraft_Carrier[0])
pyplot.show()
pyplot.imshow(Aircraft_Carrier[1])
pyplot.show()
pyplot.imshow(Aircraft_Carrier[2])
pyplot.show()


# step 1
filenames = tf.constant(Aircraft_Carrier)
labels = tf.constant([0, 1, 0, 1])

# step 1
filenames = tf.constant(['im_01.jpg', 'im_02.jpg', 'im_03.jpg', 'im_04.jpg'])
labels = tf.constant([0, 1, 0, 1])

# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

dataset = dataset.map(_parse_function)
dataset = dataset.batch(2)

# step 4: create iterator and final input tensor
iterator = dataset.make_one_shot_iterator()
images, labels = iterator.get_next()


normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))













Aircraft_Carrier = []
for filename in glob.glob('../input/vehicleImage/vehicles/Aircraft_Carrier/*.JPEG'): #assuming JPEG
    # load the image and resize

    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    Aircraft_Carrier.append(image)
      
    

for dirnames in os.walk('../input/vehicleImage/vehicles'):
    for dirname in dirnames:
        print(os.path.basename(dirname))



